# 📊 MONITOR DE INTELIGENCIA DE OPINIÓN - PROYECTO INTEGRADO
## Rubén (Organizador) & Juanes (Colaborador)
Este notebook ejecuta el pipeline completo con TODAS las visualizaciones generadas por los scripts.

In [ ]:
!pip install requests beautifulsoup4 pandas matplotlib seaborn wordcloud plotly textblob nltk
import os, pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from wordcloud import WordCloud
from textblob import TextBlob
from collections import Counter
import plotly.express as px
import plotly.graph_objects as go
for d in ['data/raw', 'data/processed', 'visualizations']: os.makedirs(d, exist_ok=True)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
print('Entorno OK.')

In [ ]:
import requests, time, random, re
from bs4 import BeautifulSoup
from datetime import datetime
import nltk, unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

class TrustpilotScraper:
    def __init__(self):
        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        self.data = []
    
    def get_reviews(self, url, pages=2):
        for p in range(1, pages + 1):
            target = url if p == 1 else f'{url}?page={p}'
            try:
                time.sleep(random.uniform(2, 4))
                r = requests.get(target, headers=self.headers, timeout=15)
                soup = BeautifulSoup(r.content, 'html.parser')
                for art in soup.find_all('article'):
                    item = self.parse(art)
                    if item:
                        self.data.append(item)
            except:
                pass
    
    def parse(self, el):
        try:
            t_el = el.select_one('[data-service-review-text-typography="true"]')
            if not t_el:
                t_el = el.select_one('p[class*="styles_reviewText"]')
            text = t_el.get_text(strip=True).replace('Ver más', '').strip() if t_el else 'Texto no disponible'
            
            stars_img = el.find('img', alt=lambda x: x and ('estrellas' in x or 'stars' in x))
            score = int(re.search(r'(\d+)', stars_img['alt']).group(1)) if stars_img else 0
            
            user = el.select_one('[data-consumer-name-typography="true"]')
            dt_el = el.find('time')
            loc = el.select_one('[data-consumer-country-typography="true"]')
            title_el = el.select_one('[data-review-title-typography="true"]')
            
            return {
                'usuario': user.get_text(strip=True) if user else 'Anonimo',
                'puntuacion': score,
                'texto_comentario': text,
                'fecha': dt_el['datetime'][:10] if dt_el else datetime.now().strftime('%Y-%m-%d'),
                'ubicacion': loc.get_text(strip=True) if loc else 'Desconocida',
                'titulo': title_el.get_text(strip=True) if title_el else 'Sin titulo'
            }
        except:
            return None

class TextPreprocessor:
    def __init__(self):
        try:
            nltk.download('punkt', quiet=True)
            nltk.download('stopwords', quiet=True)
        except:
            pass
        self.stop = set(stopwords.words('spanish'))
        self.stop.update(['amazon', 'si', 'no', 'mas', 'cada', 'hace', 'producto', 'pedido'])
    
    def clean(self, text):
        if not isinstance(text, str) or text.lower() in ['texto no disponible', 'nan', '']:
            return ''
        text = text.lower()
        text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
        text = re.sub(r'[^\w\s]', ' ', text)
        tokens = word_tokenize(text)
        return ' '.join([t for t in tokens if len(t) > 2 and t not in self.stop])

print('Clases definidas correctamente.')

## 🔄 1. Pipeline de Extracción y Limpieza

In [ ]:
scraper = TrustpilotScraper()
scraper.get_reviews('https://es.trustpilot.com/review/www.amazon.es', pages=2)
df = pd.DataFrame(scraper.data)
df.to_csv('data/raw/dataset_raw.csv', index=False)

prep = TextPreprocessor()

def combine(r):
    t = str(r['titulo'])
    c = str(r['texto_comentario'])
    if t.lower() in ['sin titulo', 'nan']:
        return c
    if c.lower() == 'texto no disponible':
        return t
    return f'{t}. {c}'

df['texto_clean_nlp'] = df.apply(lambda r: prep.clean(combine(r)), axis=1)
df['polarity'] = df['texto_clean_nlp'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
df['sentiment_label'] = df['polarity'].apply(lambda x: 'Positivo' if x>0.1 else ('Negativo' if x<-0.1 else 'Neutral'))
df['num_palabras'] = df['texto_clean_nlp'].str.split().str.len()
df['longitud_comentario'] = df['texto_comentario'].fillna('').apply(len)
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
df.to_csv('data/processed/reviews_with_sentiment.csv', index=False)

print(f'Pipeline finalizado: {len(df)} reseñas procesadas.')
df.head()

## 🎨 2. Visualización Completa - TODAS las Gráficas

In [ ]:
colors = {'Positivo': '#2ecc71', 'Neutral': '#95a5a6', 'Negativo': '#e74c3c'}
V = 'visualizations/'

# 1. WORDCLOUD
full_text = ' '.join(df['texto_clean_nlp'].dropna())
wc = WordCloud(width=1600, height=800, background_color='white', colormap='viridis').generate(full_text)
plt.figure(figsize=(15, 8))
plt.imshow(wc)
plt.axis('off')
plt.title('Nube de Palabras - Corpus Completo', fontsize=20)
plt.savefig(V+'wordcloud.png', dpi=150, bbox_inches='tight')
plt.show()

# 2. DISTRIBUCIÓN DE SENTIMIENTOS
counts = df['sentiment_label'].value_counts()
plt.figure(figsize=(10, 10))
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', 
        colors=[colors[l] for l in counts.index], startangle=90, textprops={'fontsize': 14})
plt.title('Distribución de Sentimientos', fontsize=18)
plt.savefig(V+'distribucion_sentimientos.png', dpi=150, bbox_inches='tight')
plt.show()

# 3. EVOLUCIÓN TEMPORAL
if 'fecha' in df.columns:
    df_temp = df.groupby(df['fecha'].dt.to_period('D'))['polarity'].mean().reset_index()
    df_temp['fecha'] = df_temp['fecha'].dt.to_timestamp()
    plt.figure(figsize=(14, 6))
    plt.plot(df_temp['fecha'], df_temp['polarity'], marker='o', linewidth=2, markersize=6)
    plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    plt.title('Evolución Temporal del Sentimiento', fontsize=16)
    plt.xlabel('Fecha')
    plt.ylabel('Polaridad Promedio')
    plt.grid(True, alpha=0.3)
    plt.savefig(V+'evolucion_temporal.png', dpi=150, bbox_inches='tight')
    plt.show()

# 4. HEATMAP PALABRAS POR SENTIMIENTO
hm_data = []
for s in ['Positivo', 'Negativo', 'Neutral']:
    stxt = ' '.join(df[df['sentiment_label']==s]['texto_clean_nlp'])
    for w, f in Counter(stxt.split()).most_common(5):
        hm_data.append({'Sentimiento':s, 'Palabra':w, 'Frecuencia':f})

if hm_data:
    df_hm = pd.DataFrame(hm_data).pivot_table(index='Sentimiento', columns='Palabra', values='Frecuencia', fill_value=0)
    plt.figure(figsize=(14, 7))
    sns.heatmap(df_hm, annot=True, fmt='.0f', cmap='YlGnBu', cbar_kws={'label': 'Frecuencia'})
    plt.title('Mapa de Calor: Top 5 Palabras por Sentimiento', fontsize=16)
    plt.savefig(V+'heatmap_palabras_sentimiento.png', dpi=150, bbox_inches='tight')
    plt.show()

# 5. PUNTUACIÓN VS SENTIMIENTO
plt.figure(figsize=(12, 7))
for label, color in colors.items():
    subset = df[df['sentiment_label'] == label]
    plt.scatter(subset['puntuacion'], subset['polarity'], c=color, label=label, alpha=0.6, s=100)
plt.xlabel('Puntuación (Estrellas)', fontsize=14)
plt.ylabel('Polaridad NLP', fontsize=14)
plt.title('Correlación: Puntuación Manual vs Sentimiento NLP', fontsize=16)
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig(V+'puntuacion_vs_sentimiento.png', dpi=150, bbox_inches='tight')
plt.show()

# 6. TOP 10 PALABRAS
all_words = Counter(' '.join(df['texto_clean_nlp'].dropna()).split()).most_common(10)
words, freqs = zip(*all_words)
plt.figure(figsize=(12, 7))
plt.barh(words, freqs, color='steelblue')
plt.xlabel('Frecuencia', fontsize=14)
plt.title('Top 10 Palabras Más Frecuentes', fontsize=16)
plt.gca().invert_yaxis()
plt.savefig(V+'top10_palabras.png', dpi=150, bbox_inches='tight')
plt.show()

# 7. TOP USUARIOS
top_users = df['usuario'].value_counts().head(10)
plt.figure(figsize=(12, 7))
plt.barh(top_users.index, top_users.values, color='coral')
plt.xlabel('Número de Reseñas', fontsize=14)
plt.title('Top 10 Usuarios Más Activos', fontsize=16)
plt.gca().invert_yaxis()
plt.savefig(V+'top_usuarios.png', dpi=150, bbox_inches='tight')
plt.show()

# 8. SCATTER LONGITUD VS POLARIDAD
plt.figure(figsize=(12, 7))
plt.scatter(df['longitud_comentario'], df['polarity'], 
            c=df['sentiment_label'].map(colors), alpha=0.5, s=80)
plt.xlabel('Longitud del Comentario (caracteres)', fontsize=14)
plt.ylabel('Polaridad', fontsize=14)
plt.title('Relación: Longitud del Comentario vs Polaridad', fontsize=16)
plt.grid(True, alpha=0.3)
plt.savefig(V+'scatter_longitud_polaridad.png', dpi=150, bbox_inches='tight')
plt.show()

print('TODAS las visualizaciones generadas en:', V)

## 📊 3. INFORME EJECUTIVO COMPLETO

In [ ]:
fig = plt.figure(figsize=(20, 15))
plt.suptitle('REPORTE EJECUTIVO COMPLETO - Monitor de Inteligencia de Opinión', 
             fontsize=26, fontweight='bold')

# Subplot 1: WordCloud
plt.subplot(2, 2, 1)
plt.imshow(wc)
plt.axis('off')
plt.title('Nube de Palabras', fontsize=18)

# Subplot 2: Distribución Sentimientos
plt.subplot(2, 2, 2)
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', 
        colors=[colors[l] for l in counts.index])
plt.title('Distribución de Sentimientos', fontsize=18)

# Subplot 3: Engagement por Sentimiento
plt.subplot(2, 2, 3)
sns.boxplot(x='sentiment_label', y='num_palabras', data=df, palette=colors)
plt.title('Engagement: Palabras por Sentimiento', fontsize=18)
plt.xlabel('Sentimiento')
plt.ylabel('Número de Palabras')

# Subplot 4: Distribución de Polaridad
plt.subplot(2, 2, 4)
sns.histplot(df['polarity'], kde=True, color='purple', bins=30)
plt.axvline(x=0, color='red', linestyle='--', linewidth=2)
plt.title('Distribución de Polaridad', fontsize=18)
plt.xlabel('Polaridad')
plt.ylabel('Frecuencia')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(V+'informe_ejecutivo_completo.png', dpi=150, bbox_inches='tight')
plt.show()

## 🌐 4. Dashboard Interactivo con Plotly

In [ ]:
fig = px.scatter(df, x='puntuacion', y='polarity', color='sentiment_label', 
                 color_discrete_map=colors, 
                 title='Dashboard Interactivo: Puntuación vs Polaridad',
                 hover_data=['usuario', 'fecha', 'texto_comentario'], 
                 size='num_palabras')
fig.update_layout(height=600, font=dict(size=14))
fig.write_html(V+'dashboard_interactivo.html')
fig.show()

print('ANÁLISIS COMPLETO FINALIZADO - Todas las gráficas generadas correctamente.')